In [ ]:

#VisualPulse
#Devin Holz, Troy Cunningham, Cada Kato

import numpy as np
import cv2
import sys

#initialize webcam capture
#initializes video capture using the default webcam
cap = cv2.VideoCapture(0)  

#sets capture properties to the size 640 by 480 in pixels
#Set video size to 640 by 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

#Retrieves the current frame width and height of the webcam feed and prints them
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Video size: ", frame_width, "x", frame_height)

#image properties
FrameRate = 30          #sets frame rate of video to 30 frames per second
videoChannels = 3       #assumes video is in color - RGB - so the number is 3 for the rgb
levels = 3              #number of levels for gaussian pyramid ?
minFrequency = 1.0      #bandpass filter frequency range that will detect heart rate-related frequency ranges
maxFrequency = 2.0      #same as min frequency ^
bufferSize = 30         #number of frames in buffer that is currently in process
bufferIndex = 0         #?
alpha = 50              #scaling factor applied to frame that will amplify the subtle changes in code

#creates a Gaussian pyramid, which is a series of progressively smaller 
#images (by downsampling) that helps capture image features at different scales to then capture more detail.
def buildGauss(frame, levels):
    pyramid = [frame]
    for level in range(levels):
        frame = cv2.pyrDown(frame)
        pyramid.append(frame)
    return pyramid

#reconstructs an image from a given pyramid level by progressively 
#upsampling (increasing the resolution) the image back to its original size.
def reconstructFrame(pyramid, index, levels):
    filteredFrame = pyramid[index]
    for level in range(levels):
        filteredFrame = cv2.pyrUp(filteredFrame)
    filteredFrame = filteredFrame[:frame_height, :frame_width]
    return filteredFrame

# Initialize Gaussian Pyramid                                        
firstFrame = np.zeros((frame_height, frame_width, videoChannels))  #initialize as black img
firstGauss = buildGauss(firstFrame, levels+1)[levels]              #gaussian pyramid of first frame
    #holds buffer of frames processed using gaussian pyramids
videoGauss = np.zeros((bufferSize, firstGauss.shape[0], firstGauss.shape[1], videoChannels))
fourierTransformAvg = np.zeros((bufferSize)) #stores fourier transforms of frames for further processing

# Bandpass Filter for Specified Frequencies

#Creates an array of frequency values based on the frame rate and buffer size. 
#It essentially maps the frame index to corresponding frequency values.
frequencies = (1.0*FrameRate) * np.arange(bufferSize) / (1.0*bufferSize)
#A boolean array that identifies which frequencies fall within the 
#desired heart rate frequency range (1-2 Hz, corresponding to 60-120 bpm
mask = (frequencies >= minFrequency) & (frequencies <= maxFrequency)

# Heart Rate Calculation Variables
bpmCalculationFrequency = 15            #how often the heart rate is calculated - every 15 frames
bpmBufferIndex = 0                      #index to store frames for heart rate analysis
bpmBufferSize = 10                      #size of buffer for storing frames
bpmBuffer = np.zeros((bpmBufferSize))   #buffer holding the frames used to calculate heart rate
previouspeaktime = 0      #the last detected peak time
beatspermin = 0           #variable need to store the bpm

#Text Properties 
#displays text on video frames
font = cv2.FONT_HERSHEY_SIMPLEX
loadingTextLocation = (20, 30)
bpmTextLocation = (frame_width//2 + 5, 30)
fontScale = 1
fontColor = (255,255,255)
lineType = 2
boxColor = (0, 255, 0)
boxWeight = 3




def calculate_beatspermin(bpmBuffer, frameRate):
    # Detect peaks in the time-domain signal
    peaks, _ = find_peaks(bpmBuffer, distance=frameRate*0.5)  # Ensure peaks are at least 0.5 seconds apart
               #find_peaks identifies peaks in 1D data       
               #distance converts time b/w two detected peaks into 0.5 second/0.5 secs apart
               #bpmBuffer holds the frames that calculate heart rate

    
    # Calculate the time differences between peaks
    if len(peaks) >= 2:  #len returns the length of an object. in this case length of the peaks
        peak_intervals = np.diff(peaks) / frameRate  # Time interval between peaks (in seconds)
        #we divide peaks/framerates to essentially convert our frame index into time values
    
        
        # Calculates avg. interval between peaks (seconds)
        avg_interval = np.mean(peak_intervals)
        
        # Calculate BPM: BPM = 60/avg. interval (in seconds)
        bpm = 60 / avg_interval
    else:
        bpm = 0  # Not enough peaks detected for bpm calculation
        
    return bpm




def main(bufferIndex):               #starts main loop for processing frames from webcam
    
    while(cap.isOpened()): 

        #reads frame from camera
        ret, frame = cap.read()      #captures the current frame from webcam

        if ret == True:
            
            # Construct Gaussian Pyramid
            
            #If the frame is successfully read:
            #gaussian pyramid is built for current frame.
            #fourier Transform of the Gaussian frames is calculated 
            #(this converts the signal into the frequency domain)
            videoGauss[bufferIndex] = buildGauss(frame, levels+1)[levels]
            fourierTransform = np.fft.fft(videoGauss, axis=0)
            

            # Bandpass Filter
            #This applies the bandpass filter: 
            #any frequencies that are not within the desired range (1-2 Hz) are set to zero.
            fourierTransform[mask == False] = 0

            # Amplify
            #inverse fourier transform, which converts frequency domain to time domain
            #result is amplified by multiplying by alpga to highlight to subtle changes
            filtered = np.real(np.fft.ifft(fourierTransform, axis=0))
            filtered = filtered * alpha

            # Reconstruct Resulting Frame
            #filt. frame is reconstructed and combined with original frame to create output frame
            filteredFrame = reconstructFrame(filtered, bufferIndex, levels)
            outputFrame = frame + filteredFrame
            outputFrame = cv2.convertScaleAbs(outputFrame) #ensures pixel values are in valid range for display

            #buffer index is incremented and wrapped around to stay within the buffer size.
            bufferIndex = (bufferIndex + 1) % bufferSize
            

            # Display the resulting frame
            #processed frame is displayed in a window titled "VisualPulse".
            cv2.imshow('VisualPulse', outputFrame)


            


            
            # Press Q on keyboard to exit
            if cv2.waitKey(20) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()


main(bufferIndex)
Video size:  640 x 480
 
